
# Limpeza e pré Precessamento de Dados


**Dados recolhido DATASUS**


Nesta página, nós primeiramente unimos os dados brutos do DATASUS sobre a COVID-19 dos anos 2021, 2022 e 2023 em um único conjunto. Em seguida, realizamos um processo de pré-processamento para corrigir possíveis inconsistências e erros nesse conjunto de dados consolidado. Essa etapa foi fundamental para garantir a qualidade e confiabilidade das informações ao longo desses três anos.



Durante o processo de limpeza de dados, identificamos e corrigimos erros, como valores ausentes, duplicados ou inconsistentes. Além disso, realizamos a padronização de formatos e a validação de informações para assegurar a integridade dos dados. Esse procedimento foi essencial para garantir que os dados estivessem precisos e prontos para análise.

In [497]:
# Handle table-like data and matrices :
import numpy as np
import pandas as pd
import math
import itertools
import datetime as datetime
from datetime import date
import os



#Modelling Helpers :
#from sklearn.preprocessing import Imputer , Normalizer , scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score


# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
#import missingno as msno



# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
plt.style.use('fivethirtyeight')
sns.set(context="notebook", palette="dark", style = 'whitegrid' , color_codes=True)


In [498]:
# Center all plots
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""");


# Make Visualizations better
params = {
    'axes.labelsize': "large",
    'xtick.labelsize': 'x-large',
    'legend.fontsize': 20,
    'figure.dpi': 150,
    'figure.figsize': [25, 7]
}
plt.rcParams.update(params)

In [499]:
# Leia o arquivo CSV e ignore os warnings
import warnings
warnings.filterwarnings('ignore')

df_21=pd.read_csv('INFLUD21-01-05-2023.csv', sep=';')
df_22 = pd.read_csv('INFLUD22-03-04-2023.csv', sep=';', skiprows=[12347])
df_23=pd.read_csv('INFLUD23-19-06-2023.csv', sep=';')

In [500]:
# Mostra as cinco primeiras linhas do DataFrame df_1
#df_21.head(3)
df_23['CS_GESTANT'].unique()


array([ 5.,  6.,  3.,  9.,  4.,  1.,  2.,  0., nan])

Data set  2021,2022,2023----------------------------------

In [501]:
# Configurar pandas para exibir todas as colunas
pd.set_option('display.max_columns', None)

In [502]:
df1_selected = df_21[['DT_SIN_PRI','SEM_PRI','SG_UF_NOT','ID_MUNICIP','CO_MUN_NOT',
                    'CS_SEXO','DT_NASC','NU_IDADE_N','TP_IDADE','CS_GESTANT','CS_RACA','CS_ESCOL_N',
                   'CS_ZONA','NOSOCOMIAL','DISPNEIA','DIARREIA','DESC_RESP','PERD_OLFT','PERD_PALA','PNEUMOPATI','FADIGA','TOSSE','CARDIOPATI',
                   'SIND_DOWN','ASMA','DIABETES','NEUROLOGIC','RENAL','OBESIDADE','VACINA_COV','HOSPITAL','DT_INTERNA','UTI',
                    'EVOLUCAO','DT_EVOLUCA','DOSE_1_COV','DOSE_2_COV','FATOR_RISC',
                   ]]

In [503]:
df2_selected = df_22[['DT_SIN_PRI','SEM_PRI','SG_UF_NOT','ID_MUNICIP','CO_MUN_NOT',
                    'CS_SEXO','DT_NASC','NU_IDADE_N','TP_IDADE','CS_GESTANT','CS_RACA','CS_ESCOL_N',
                   'CS_ZONA','NOSOCOMIAL','DISPNEIA','DIARREIA','DESC_RESP','PERD_OLFT','PERD_PALA','PNEUMOPATI','FADIGA','TOSSE','CARDIOPATI',
                   'SIND_DOWN','ASMA','DIABETES','NEUROLOGIC','RENAL','OBESIDADE','VACINA_COV','HOSPITAL','DT_INTERNA','UTI',
                    'EVOLUCAO','DT_EVOLUCA','DOSE_1_COV','DOSE_2_COV','FATOR_RISC',
                   ]]

In [504]:
df3_selected = df_23[['DT_SIN_PRI','SEM_PRI','SG_UF_NOT','ID_MUNICIP','CO_MUN_NOT',
                    'CS_SEXO','DT_NASC','NU_IDADE_N','TP_IDADE','CS_GESTANT','CS_RACA','CS_ESCOL_N',
                   'CS_ZONA','NOSOCOMIAL','DISPNEIA','DIARREIA','DESC_RESP','PERD_OLFT','PERD_PALA','PNEUMOPATI','FADIGA','TOSSE','CARDIOPATI',
                   'SIND_DOWN','ASMA','DIABETES','NEUROLOGIC','RENAL','OBESIDADE','VACINA_COV','HOSPITAL','DT_INTERNA','UTI',
                    'EVOLUCAO','DT_EVOLUCA','DOSE_1_COV','DOSE_2_COV','FATOR_RISC',
                   ]]

In [505]:
# Adicionar coluna de indicativo de ano para cada DataFrame
df1_selected['Ano'] = 2021
df2_selected['Ano'] = 2022
df3_selected['Ano'] = 2023

# Concatenar os DataFrames em um único DataFrame
covidLong = pd.concat([df1_selected, df2_selected, df3_selected])

# Mostrar o DataFrame resultante
print(covidLong)

       DT_SIN_PRI  SEM_PRI SG_UF_NOT             ID_MUNICIP  CO_MUN_NOT  \
0      06/01/2021        1        BA             ALAGOINHAS    290070.0   
1      03/01/2021        1        BA                ITABUNA    291480.0   
2      07/01/2021        1        PR               CURITIBA    410690.0   
3      05/01/2021        1        PR               LONDRINA    411370.0   
4      04/01/2021        1        RS            DOM PEDRITO    430660.0   
...           ...      ...       ...                    ...         ...   
12383  01/02/2023        5        SP  SAO JOSE DO RIO PRETO    354980.0   
12384  27/01/2023        4        RO                 CACOAL    110004.0   
12385  16/02/2023        7        SP               CAMPINAS    350950.0   
12386  13/05/2023       19        SE                ARACAJU    280030.0   
12387  19/05/2023       20        RS                    NaN         NaN   

      CS_SEXO     DT_NASC  NU_IDADE_N  TP_IDADE  CS_GESTANT  CS_RACA  \
0           M  07/09/1951  

In [506]:
covidLong.head(3)

,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,CS_ZONA,NOSOCOMIAL,DISPNEIA,DIARREIA,DESC_RESP,PERD_OLFT,PERD_PALA,PNEUMOPATI,FADIGA,TOSSE,CARDIOPATI,SIND_DOWN,ASMA,DIABETES,NEUROLOGIC,RENAL,OBESIDADE,VACINA_COV,HOSPITAL,DT_INTERNA,UTI,EVOLUCAO,DT_EVOLUCA,DOSE_1_COV,DOSE_2_COV,FATOR_RISC,Ano
0,06/01/2021,1,BA,ALAGOINHAS,290070.0,M,07/09/1951,69.0,3.0,6.0,1.0,9.0,1.0,NaN,1.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,06/01/2021,1.0,2.0,10/01/2021,NaN,NaN,2.0,2021
1,03/01/2021,1,BA,ITABUNA,291480.0,M,19/07/1931,89.0,3.0,6.0,9.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,03/01/2021,1.0,2.0,05/01/2021,NaN,NaN,2.0,2021
2,07/01/2021,1,PR,CURITIBA,410690.0,F,07/11/1969,51.0,3.0,5.0,9.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,10/01/2021,1.0,1.0,18/01/2021,NaN,NaN,1.0,2021


In [507]:
print(covidLong.columns)

Index(['DT_SIN_PRI', 'SEM_PRI', 'SG_UF_NOT', 'ID_MUNICIP', 'CO_MUN_NOT',
       'CS_SEXO', 'DT_NASC', 'NU_IDADE_N', 'TP_IDADE', 'CS_GESTANT', 'CS_RACA',
       'CS_ESCOL_N', 'CS_ZONA', 'NOSOCOMIAL', 'DISPNEIA', 'DIARREIA',
       'DESC_RESP', 'PERD_OLFT', 'PERD_PALA', 'PNEUMOPATI', 'FADIGA', 'TOSSE',
       'CARDIOPATI', 'SIND_DOWN', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'RENAL',
       'OBESIDADE', 'VACINA_COV', 'HOSPITAL', 'DT_INTERNA', 'UTI', 'EVOLUCAO',
       'DT_EVOLUCA', 'DOSE_1_COV', 'DOSE_2_COV', 'FATOR_RISC', 'Ano'],
      dtype='object')


In [508]:
# Fazer uma cópia do DataFrame
covidLonga_copia = covidLong.copy()

# Limpeza e Pré-processamento de dados 

## 1- Identificação de Dados Ausentes (Missing Data):


In [509]:
# Identificar valores ausentes
valores_ausentes = covidLong.isna()

# Mostrar os valores ausentes
print(valores_ausentes)

       DT_SIN_PRI  SEM_PRI  SG_UF_NOT  ID_MUNICIP  CO_MUN_NOT  CS_SEXO  \
0           False    False      False       False       False    False   
1           False    False      False       False       False    False   
2           False    False      False       False       False    False   
3           False    False      False       False       False    False   
4           False    False      False       False       False    False   
...           ...      ...        ...         ...         ...      ...   
12383       False    False      False       False       False    False   
12384       False    False      False       False       False    False   
12385       False    False      False       False       False    False   
12386       False    False      False       False       False    False   
12387       False    False      False        True        True     True   

       DT_NASC  NU_IDADE_N  TP_IDADE  CS_GESTANT  CS_RACA  CS_ESCOL_N  \
0        False       False     False  

### Filtrando os casos relacinado a mais de 3 semanas com sintomas 


In [510]:
covidLong

,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,CS_ZONA,NOSOCOMIAL,DISPNEIA,DIARREIA,DESC_RESP,PERD_OLFT,PERD_PALA,PNEUMOPATI,FADIGA,TOSSE,CARDIOPATI,SIND_DOWN,ASMA,DIABETES,NEUROLOGIC,RENAL,OBESIDADE,VACINA_COV,HOSPITAL,DT_INTERNA,UTI,EVOLUCAO,DT_EVOLUCA,DOSE_1_COV,DOSE_2_COV,FATOR_RISC,Ano
0,06/01/2021,1,BA,ALAGOINHAS,290070.0,M,07/09/1951,69.0,3.0,6.0,1.0,9.0,1.0,NaN,1.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,06/01/2021,1.0,2.0,10/01/2021,NaN,NaN,2.0,2021
1,03/01/2021,1,BA,ITABUNA,291480.0,M,19/07/1931,89.0,3.0,6.0,9.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,03/01/2021,1.0,2.0,05/01/2021,NaN,NaN,2.0,2021
2,07/01/2021,1,PR,CURITIBA,410690.0,F,07/11/1969,51.0,3.0,5.0,9.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,10/01/2021,1.0,1.0,18/01/2021,NaN,NaN,1.0,2021
3,05/01/2021,1,PR,LONDRINA,411370.0,M,12/06/1963,57.0,3.0,6.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,05/01/2021,2.0,3.0,06/01/2021,NaN,NaN,1.0,2021
4,04/01/2021,1,RS,DOM PEDRITO,430660.0,M,03/03/1939,81.0,3.0,6.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,11/01/2021,1.0,2.0,15/01/2021,NaN,NaN,1.0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12383,01/02/2023,5,SP,SAO JOSE DO RIO PRETO,354980.0,F,19/05/2022,9.0,2.0,6.0,2.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,02/02/2023,2.0,1.0,05/02/2023,NaN,NaN,1.0,2023
12384,27/01/2023,4,RO,CACOAL,110004.0,F,01/04/2022,9.0,2.0,6.0,4.0,NaN,1.0,2.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,28/01/2023,1.0,1.0,13/02/2023,NaN,NaN,2.0,2023
12385,16/02/2023,7,SP,CAMPINAS,350950.0,M,14/02/2016,7.0,3.0,6.0,9.0,9.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,17/02/2023,1.0,1.0,21/02/2023,NaN,NaN,1.0,2023
12386,13/05/2023,19,SE,ARACAJU,280030.0,F,16/12/2020,2.0,3.0,6.0,4.0,9.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,17/05/2023,2.0,NaN,NaN,07/02/2023,NaN,2.0,2023


In [511]:
import pandas as pd



# Definir o valor para filtragem
valor_filtrado = 3

# Filtrar o DataFrame com base no valor da coluna
covidLong = covidLong[covidLong['SEM_PRI'] >= valor_filtrado]

# Mostrar o DataFrame filtrado
print(covidLong)

       DT_SIN_PRI  SEM_PRI SG_UF_NOT             ID_MUNICIP  CO_MUN_NOT  \
142    17/01/2021        3        PR               LONDRINA    411370.0   
150    02/02/2021        5        MG             UBERLANDIA    317020.0   
199    18/01/2021        3        SC    SAO MIGUEL DO OESTE    421720.0   
214    17/01/2021        3        SP              PENAPOLIS    353730.0   
241    20/01/2021        3        SP              SAO PAULO    355030.0   
...           ...      ...       ...                    ...         ...   
12383  01/02/2023        5        SP  SAO JOSE DO RIO PRETO    354980.0   
12384  27/01/2023        4        RO                 CACOAL    110004.0   
12385  16/02/2023        7        SP               CAMPINAS    350950.0   
12386  13/05/2023       19        SE                ARACAJU    280030.0   
12387  19/05/2023       20        RS                    NaN         NaN   

      CS_SEXO     DT_NASC  NU_IDADE_N  TP_IDADE  CS_GESTANT  CS_RACA  \
142         M  18/10/1956  

## 2-Tratamento de Dados Ausentes:


In [513]:
# Preencher os valores ausentes com -1
covidLong = covidLong.fillna(-1)
covidLong.head(2)

,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,CS_ZONA,NOSOCOMIAL,DISPNEIA,DIARREIA,DESC_RESP,PERD_OLFT,PERD_PALA,PNEUMOPATI,FADIGA,TOSSE,CARDIOPATI,SIND_DOWN,ASMA,DIABETES,NEUROLOGIC,RENAL,OBESIDADE,VACINA_COV,HOSPITAL,DT_INTERNA,UTI,EVOLUCAO,DT_EVOLUCA,DOSE_1_COV,DOSE_2_COV,FATOR_RISC,Ano
142,17/01/2021,3,PR,LONDRINA,411370.0,M,18/10/1956,64.0,3.0,6.0,1.0,9.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,17/01/2021,2.0,2.0,18/01/2021,-1,-1,2.0,2021
150,02/02/2021,5,MG,UBERLANDIA,317020.0,M,13/05/1936,84.0,3.0,6.0,4.0,9.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,-1.0,2.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,17/02/2021,2.0,1.0,03/03/2021,27/03/2021,-1,2.0,2021


In [514]:
# Definir a nova ordem das colunas
nova_ordem_colunas = ['DT_SIN_PRI','SEM_PRI','CS_SEXO', 'NU_IDADE_N', 'CS_RACA', 'CS_ESCOL_N', 'SG_UF_NOT', 'ID_MUNICIP', 'CS_ZONA']

# Obter a lista de colunas restantes na ordem original
colunas_restantes = [coluna for coluna in covidLong.columns if coluna not in nova_ordem_colunas]

# Reorganizar as colunas do DataFrame
covidLong = covidLong[nova_ordem_colunas + colunas_restantes]

# Mostrar o DataFrame com as colunas reorganizadas
covidLong.head(2)

,DT_SIN_PRI,SEM_PRI,CS_SEXO,NU_IDADE_N,CS_RACA,CS_ESCOL_N,SG_UF_NOT,ID_MUNICIP,CS_ZONA,CO_MUN_NOT,DT_NASC,TP_IDADE,CS_GESTANT,NOSOCOMIAL,DISPNEIA,DIARREIA,DESC_RESP,PERD_OLFT,PERD_PALA,PNEUMOPATI,FADIGA,TOSSE,CARDIOPATI,SIND_DOWN,ASMA,DIABETES,NEUROLOGIC,RENAL,OBESIDADE,VACINA_COV,HOSPITAL,DT_INTERNA,UTI,EVOLUCAO,DT_EVOLUCA,DOSE_1_COV,DOSE_2_COV,FATOR_RISC,Ano
142,17/01/2021,3,M,64.0,1.0,9.0,PR,LONDRINA,1.0,411370.0,18/10/1956,3.0,6.0,2.0,1.0,2.0,1.0,2.0,2.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,17/01/2021,2.0,2.0,18/01/2021,-1,-1,2.0,2021
150,02/02/2021,5,M,84.0,4.0,9.0,MG,UBERLANDIA,1.0,317020.0,13/05/1936,3.0,6.0,2.0,1.0,2.0,1.0,2.0,2.0,-1.0,2.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,17/02/2021,2.0,1.0,03/03/2021,27/03/2021,-1,2.0,2021


In [515]:
covidLong = covidLong.drop([ 'CO_MUN_NOT','DT_NASC'], axis=1)
# excluir coluna-DT_SIN_PRI', 'CO_MUN_NOT','DT_NASC'

In [516]:
#covidLong = covidLong.drop(['TP_IDADE'], axis=1)


In [443]:
covidLong.head(2)

,DT_SIN_PRI,SEM_PRI,CS_SEXO,NU_IDADE_N,CS_RACA,CS_ESCOL_N,SG_UF_NOT,ID_MUNICIP,CS_ZONA,CS_GESTANT,NOSOCOMIAL,DISPNEIA,DIARREIA,DESC_RESP,PERD_OLFT,PERD_PALA,PNEUMOPATI,FADIGA,TOSSE,CARDIOPATI,SIND_DOWN,ASMA,DIABETES,NEUROLOGIC,RENAL,OBESIDADE,VACINA_COV,HOSPITAL,DT_INTERNA,UTI,EVOLUCAO,DT_EVOLUCA,DOSE_1_COV,DOSE_2_COV,FATOR_RISC,Ano
142,17/01/2021,3,M,64.0,1.0,9.0,PR,LONDRINA,1.0,6.0,2.0,1.0,2.0,1.0,2.0,2.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,17/01/2021,2.0,2.0,18/01/2021,-1,-1,2.0,2021
150,02/02/2021,5,M,84.0,4.0,9.0,MG,UBERLANDIA,1.0,6.0,2.0,1.0,2.0,1.0,2.0,2.0,-1.0,2.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,17/02/2021,2.0,1.0,03/03/2021,27/03/2021,-1,2.0,2021


In [444]:
covidLong['CS_GESTANT'].unique()

array([ 6.,  5.,  9.,  1.,  2.,  3.,  4.,  0., -1.])

## 3-Tratamento de Valores Atípicos (Outliers):

In [517]:
covidLong['NU_IDADE_N'].unique()

array([ 64.,  84.,  85.,  59.,  74.,  89.,  34.,  81.,  72.,  77.,  69.,
        78.,  63.,  49.,  47.,  92.,  33.,  58.,  65.,  60.,  14.,  55.,
         2.,   3.,  66.,  36.,  54.,  82.,  39.,  73.,  51.,  71.,  62.,
        50.,  86.,  79.,  83.,   0.,  91.,   1.,   6.,  52.,  45.,  28.,
        68.,  42.,  27.,  48.,  38.,  67.,  12.,  37.,   4.,  95.,  40.,
        61.,   7.,  75.,  76.,  35.,  56.,  57.,  24.,  99.,  88.,  70.,
         8.,  22.,  20.,  32.,  41.,   5.,  30.,  94.,  87.,  26.,  53.,
        80.,  21.,  93.,  43.,  25.,  46.,  44.,   9.,  18.,  19.,  17.,
        90.,  29.,  15.,  31.,  13.,  98.,  96.,  11.,  23., 102.,  10.,
        16., 100.,  97., 101., 104., 106., 105., 103.,  -1., 116.])

In [518]:
# eliminar idade maior que 105 anos 
# Eliminar linhas onde o valor da coluna 'Coluna' seja igual a 3
covidLong = covidLong.drop(covidLong[covidLong['NU_IDADE_N']>= 106].index)

In [519]:
valores_unicos = np.unique(covidLong['NU_IDADE_N'])
valores_unicos_ordenados = np.sort(valores_unicos)
valores_unicos_ordenados

array([ -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,
        10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,
        21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,
        32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,
        43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,
        54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,
        65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,
        76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,
        87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,
        98.,  99., 100., 101., 102., 103., 104., 105.])

In [646]:
# Substitua 'seu_nome_de_arquivo.csv' pelo nome desejado para o arquivo CSV
covidLong.to_csv('covidLong.csv', index=False)

## 4- Variavel target

Se o seu objetivo é analisar os fatores associados ao desenvolvimento da Síndrome da Covid longa, uma possibilidade seria usar a variável EVOLUCAO, que indica a evolução do caso. Nesse caso, você poderia criar uma variável binária indicando se o paciente teve uma evolução para a Covid longa (1) ou não (0).


A variável "Evolução do caso" (EVOLUCAO) pode ser uma variável target porque ela representa o desfecho ou resultado do caso de Covid-19 para um paciente específico.A variável "Evolução do caso" (EVOLUCAO) pode ser uma variável target porque ela representa o desfecho ou resultado do caso de Covid-19 para um paciente específico. No contexto da sua análise, a escolha de EVOLUCAO como variável target pode ser apropriada por algumas razões:



**Relevância Clínica:**
A evolução do caso é uma medida clínica importante que indica se o paciente se recuperou (Cura), teve um desfecho fatal (Óbito) ou outros desfechos específicos. Essa informação pode ser valiosa para entender a gravidade e o curso clínico da doença.



**Consequências de Longo Prazo:**
A Síndrome da Covid longa é caracterizada pela persistência de sintomas por um período prolongado. Se a evolução do caso estiver relacionada à presença ou ausência da Síndrome da Covid longa, isso pode fornecer insights sobre os fatores associados a essa condição.


**Facilidade de Interpretação:**
A variável EVOLUCAO é uma medida direta e facilmente interpretável da evolução do caso. Cada categoria (Cura, Óbito, Óbito por outras causas, Ignorado) representa um resultado distinto.



**Possibilidade de Análise Estatística e Modelagem Preditiva:**
Ao usar EVOLUCAO como variável target, você pode realizar análises estatísticas descritivas para entender as características dos diferentes desfechos. Além disso, é possível aplicar técnicas de modelagem preditiva para prever a evolução do caso com base em outras variáveis explicativas.



In [520]:
# Assumindo que 'covidLong' é o seu DataFrame
covidLong['CovidLonga'] = (covidLong['EVOLUCAO'] != 1) & (covidLong['EVOLUCAO'] != 9)
covidLong['CovidLonga'] = covidLong['CovidLonga'].astype(int)

# Mostrar o DataFrame com a nova variável alvo
print(covidLong)

       DT_SIN_PRI  SEM_PRI CS_SEXO  NU_IDADE_N  CS_RACA  CS_ESCOL_N SG_UF_NOT  \
142    17/01/2021        3       M        64.0      1.0         9.0        PR   
150    02/02/2021        5       M        84.0      4.0         9.0        MG   
199    18/01/2021        3       M        64.0      1.0         3.0        SC   
214    17/01/2021        3       M        85.0      1.0        -1.0        SP   
241    20/01/2021        3       M        59.0      1.0         9.0        SP   
...           ...      ...     ...         ...      ...         ...       ...   
12383  01/02/2023        5       F         9.0      2.0         0.0        SP   
12384  27/01/2023        4       F         9.0      4.0        -1.0        RO   
12385  16/02/2023        7       M         7.0      9.0         9.0        SP   
12386  13/05/2023       19       F         2.0      4.0         9.0        SE   
12387  19/05/2023       20      -1        -1.0     -1.0        -1.0        RS   

                  ID_MUNICI

Essa linha de código cria uma nova coluna chamada CovidLonga no DataFrame covidLong. A explicação é a seguinte:

covidLong['EVOLUCAO'] != 1: Esta parte compara cada valor na coluna EVOLUCAO com 1. A expressão resulta em uma série de valores booleanos (True ou False), onde True indica que o valor em EVOLUCAO não é igual a 1.
covidLong['EVOLUCAO'] != 9: Similar à primeira parte, esta comparação verifica se cada valor na coluna EVOLUCAO é diferente de 9.


(covidLong['EVOLUCAO'] != 1) & (covidLong['EVOLUCAO'] != 9): O operador & realiza uma operação lógica "AND" elemento a elemento entre as duas séries booleanas resultantes das comparações anteriores. Isso significa que a condição True na nova coluna CovidLonga ocorre apenas quando ambas as condições True são atendidas (ou seja, quando EVOLUCAO não é igual a 1 e não é igual a 9).


Portanto, a nova coluna CovidLonga terá valores True nos casos em que a EVOLUCAO não for 1 e não for 9, o que sugere que a evolução não é "Cura" nem "Ignorado". Esses casos podem ser interpretados como potencialmente indicativos de Síndrome da Covid longa, conforme sua lógica específica.

Essa abordagem considera como "positivos" os casos que não resultaram em cura e não foram ignorados, deixando espaço para outras categorias possivelmente relacionadas à Covid longa.

In [521]:
covidLong['CS_GESTANT'].unique()

array([ 6.,  5.,  9.,  1.,  2.,  3.,  4.,  0., -1.])

In [647]:
# Verifique o número de linhas no DataFrame
num_linhas = covidLong.shape[0]

# Exiba o resultado
print(f"O DataFrame tem {num_linhas} linhas.")

O DataFrame tem 35466 linhas.


**Função para mapear a idade para os grupos correspondentes**

In [522]:
# Função para mapear a idade para os grupos correspondentes
def mapear_grupo_idade(idade):
    if idade in [ -9,  -6,  -5,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,]:
        return 'Grupo 0'
    elif idade in [6, 7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17]:
        return 'Grupo 1'
    elif idade in [18,  19,
        20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49]:
        return 'Grupo 2'
    elif idade in [50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65]:
        return 'Grupo 3'
    
    elif idade in [66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79]:
         return 'Grupo 4'
    elif idade in [  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105]:
        return 'Grupo 5'

# Aplicar a função aos valores da coluna 'idade' e criar a nova coluna 'grupo_idade'
covidLong['FAIXA_IDADE'] = covidLong['NU_IDADE_N'].apply(mapear_grupo_idade)

# Mostrar o DataFrame resultante
print(covidLong)

       DT_SIN_PRI  SEM_PRI CS_SEXO  NU_IDADE_N  CS_RACA  CS_ESCOL_N SG_UF_NOT  \
142    17/01/2021        3       M        64.0      1.0         9.0        PR   
150    02/02/2021        5       M        84.0      4.0         9.0        MG   
199    18/01/2021        3       M        64.0      1.0         3.0        SC   
214    17/01/2021        3       M        85.0      1.0        -1.0        SP   
241    20/01/2021        3       M        59.0      1.0         9.0        SP   
...           ...      ...     ...         ...      ...         ...       ...   
12383  01/02/2023        5       F         9.0      2.0         0.0        SP   
12384  27/01/2023        4       F         9.0      4.0        -1.0        RO   
12385  16/02/2023        7       M         7.0      9.0         9.0        SP   
12386  13/05/2023       19       F         2.0      4.0         9.0        SE   
12387  19/05/2023       20      -1        -1.0     -1.0        -1.0        RS   

                  ID_MUNICI

In [523]:
covidLong.head(2)

,DT_SIN_PRI,SEM_PRI,CS_SEXO,NU_IDADE_N,CS_RACA,CS_ESCOL_N,SG_UF_NOT,ID_MUNICIP,CS_ZONA,TP_IDADE,CS_GESTANT,NOSOCOMIAL,DISPNEIA,DIARREIA,DESC_RESP,PERD_OLFT,PERD_PALA,PNEUMOPATI,FADIGA,TOSSE,CARDIOPATI,SIND_DOWN,ASMA,DIABETES,NEUROLOGIC,RENAL,OBESIDADE,VACINA_COV,HOSPITAL,DT_INTERNA,UTI,EVOLUCAO,DT_EVOLUCA,DOSE_1_COV,DOSE_2_COV,FATOR_RISC,Ano,CovidLonga,FAIXA_IDADE
142,17/01/2021,3,M,64.0,1.0,9.0,PR,LONDRINA,1.0,3.0,6.0,2.0,1.0,2.0,1.0,2.0,2.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,17/01/2021,2.0,2.0,18/01/2021,-1,-1,2.0,2021,1,Grupo 3
150,02/02/2021,5,M,84.0,4.0,9.0,MG,UBERLANDIA,1.0,3.0,6.0,2.0,1.0,2.0,1.0,2.0,2.0,-1.0,2.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,17/02/2021,2.0,1.0,03/03/2021,27/03/2021,-1,2.0,2021,0,Grupo 5
